<a href="https://colab.research.google.com/github/CL500Coupe/data_analytics_assi1/blob/main/assi2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import sys
import os
import pandas as pd
import seaborn as sns
import pickle
import time
from sklearn.model_selection import GridSearchCV
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [46]:
url = "https://raw.githubusercontent.com/CL500Coupe/data_analytics_assi1/main/data/final_data.csv"
data = pd.read_csv(url)

In [47]:
# Convert the 'collision_date' column to datetime format
data['collision_date'] = pd.to_datetime(data['collision_date'])

# Set the 'collision_date' column as the index
data = data.set_index('collision_date')

In [48]:
# List of columns with NA values to fill with mean values of the last 7 available days
columns_with_na_7 = [
    "visib_7day_ma",
    "wdsp_7day_ma",
    "co2_7day_ma",
    "so2_7day_ma",
    "pressure_7day_ma",
    "temp_7day_ma",
    "wind_7day_ma",
]

# List of columns with NA values to fill with mean values of the last 30 available days
columns_with_na_30 = [
    "visib_30day_ma",
    "wdsp_30day_ma",
    "co2_30day_ma",
    "so2_30day_ma",
    "pressure_30day_ma",
    "temp_30day_ma",
    "wind_30day_ma",
]

# Fill NA values with the mean values of the first 7 available days
for col in columns_with_na_7:
    first_valid_index = data[col].first_valid_index()
    mean_value = data.loc[first_valid_index:first_valid_index + pd.Timedelta(days=6), col].mean()
    data[col] = data[col].fillna(mean_value, limit=6)

# Fill NA values with the mean values of the first 30 available days
for col in columns_with_na_30:
    first_valid_index = data[col].first_valid_index()
    mean_value = data.loc[first_valid_index:first_valid_index + pd.Timedelta(days=29), col].mean()
    data[col] = data[col].fillna(mean_value, limit=29)

# Display the updated DataFrame
print(data.head())



                day  year  mo  temp  dewp     slp  visib  wdsp  mxpsd  gust  \
collision_date                                                                
2012-07-01        7  2012   7  83.6  63.0  1008.9    9.7   4.1    9.9  18.1   
2012-07-02        1  2012   7  80.3  54.1  1011.6   10.0   3.8   15.0  18.1   
2012-07-03        2  2012   7  79.8  56.7  1012.8   10.0   2.9   12.0  18.1   
2012-07-04        3  2012   7  81.8  65.6  1009.1    9.2   3.6   11.1  18.1   
2012-07-05        4  2012   7  86.7  64.3  1007.4    9.4   3.8   15.0  18.1   

                ...  co2_30day_ma  so2_30day_ma  pressure_30day_ma  \
collision_date  ...                                                  
2012-07-01      ...     15.031652      1.210648         917.765355   
2012-07-02      ...     15.031652      1.210648         917.765355   
2012-07-03      ...     15.031652      1.210648         917.765355   
2012-07-04      ...     15.031652      1.210648         917.765355   
2012-07-05      ...     15

In [49]:
data.shape


(2969, 39)

In [50]:
data.head()

,day,year,mo,temp,dewp,slp,visib,wdsp,mxpsd,gust,...,co2_30day_ma,so2_30day_ma,pressure_30day_ma,temp_30day_ma,wind_30day_ma,temp_visib_interaction,wdsp_visib_interaction,prcp_temp_interaction,temp_season_interaction,wdsp_wind_interaction
collision_date,,,,,,,,,,,,,,,,,,,,,
2012-07-01,7,2012,7,83.6,63.0,1008.9,9.7,4.1,9.9,18.1,...,15.031652,1.210648,917.765355,76.092,93.001803,810.92,39.77,0.000,167.2,553.145520
2012-07-02,1,2012,7,80.3,54.1,1011.6,10.0,3.8,15.0,18.1,...,15.031652,1.210648,917.765355,76.092,93.001803,803.00,38.00,0.000,160.6,545.418750
2012-07-03,2,2012,7,79.8,56.7,1012.8,10.0,2.9,12.0,18.1,...,15.031652,1.210648,917.765355,76.092,93.001803,798.00,29.00,0.000,159.6,362.080104
2012-07-04,3,2012,7,81.8,65.6,1009.1,9.2,3.6,11.1,18.1,...,15.031652,1.210648,917.765355,76.092,93.001803,752.56,33.12,4.908,163.6,453.314999
2012-07-05,4,2012,7,86.7,64.3,1007.4,9.4,3.8,15.0,18.1,...,15.031652,1.210648,917.765355,76.092,93.001803,814.98,35.72,8669.133,173.4,479.916250


In [51]:
data.dtypes

day                          int64
year                         int64
mo                           int64
temp                       float64
dewp                       float64
slp                        float64
visib                      float64
wdsp                       float64
mxpsd                      float64
gust                       float64
max                        float64
min                        float64
prcp                       float64
fog                          int64
NUM_COLLISIONS               int64
co2                        float64
wind                       float64
so2                        float64
pressure                   float64
season                       int64
visib_7day_ma              float64
wdsp_7day_ma               float64
co2_7day_ma                float64
so2_7day_ma                float64
pressure_7day_ma           float64
temp_7day_ma               float64
wind_7day_ma               float64
visib_30day_ma             float64
wdsp_30day_ma       

In [ ]:
data.columns

Index(['day', 'year', 'mo', 'temp', 'dewp', 'slp', 'visib', 'wdsp', 'mxpsd',
       'gust', 'max', 'min', 'prcp', 'fog', 'NUM_COLLISIONS', 'co2', 'wind',
       'so2', 'pressure', 'season', 'visib_7day_ma', 'wdsp_7day_ma',
       'co2_7day_ma', 'so2_7day_ma', 'pressure_7day_ma', 'temp_7day_ma',
       'wind_7day_ma', 'visib_30day_ma', 'wdsp_30day_ma', 'co2_30day_ma',
       'so2_30day_ma', 'pressure_30day_ma', 'temp_30day_ma', 'wind_30day_ma',
       'temp_visib_interaction', 'wdsp_visib_interaction',
       'prcp_temp_interaction', 'temp_season_interaction',
       'wdsp_wind_interaction'],
      dtype='object')

In [53]:
# Check if there are any missing values in the entire DataFrame
has_missing_values = data.isna().any().any()

# Print the result
print("Are there any missing values in the DataFrame?", has_missing_values)

Are there any missing values in the DataFrame? False


In [54]:
# Select the features and target variable
X = data.drop("NUM_COLLISIONS", axis=1)  # Replace "num_collisions" with the actual target column name
y = data["NUM_COLLISIONS"]  # Replace "num_collisions" with the actual target column name

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [55]:
# Create a linear regression model
linear_regression = LinearRegression()

In [56]:
# Train the model
linear_regression.fit(X_train, y_train)

ValueError: ignored